In [38]:
import sys
dll_path = r'C:\Users\bkmz1\PycharmProjects\manipulator\dobot_lib'
sys.path.insert(1,dll_path)
import DobotDllType as dType
from math import pi
import numpy as np
import time

In [42]:
com = 'COM4'
api = dType.load(dll_path)

The dll you are using is 64-bit. In order to run smoothly, please ensure that your python environment is also 64-bit.
The python environment is： ('64bit', 'WindowsPE')


In [43]:
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:", state)
#if (state == dType.DobotConnect.DobotConnect_NoError):
    #Dobot interactive codes
#dType.DisconnectDobot(api)

Connect status: 0


In [41]:
dType.DisconnectDobot(api)

In [44]:
dType.SetQueuedCmdClear(api)
dType.SetQueuedCmdStartExec(api)

In [45]:
dType.SetHOMEParams(api, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPJointParams(api, 200, 200, 200, 200, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPCommonParams(api, 100, 100, isQueued = 1)

[4]

In [149]:
from math import dist, atan2
from itertools import combinations, islice

def coords_from_chess_pos(pos):
    # from chess pos (e.g. 'e4') into local chess coords
    # letters - xi axis, numbers - eta. Origin at center of a1
    assert len(pos) == 2
    if isinstance(pos, str):
        char, num = pos.lower()
        xi = ord(char) - ord('a')
        eta = int(num) - 1
        return np.array([xi, eta])
    else:
        return np.array(pos)
    

def find_intersections(o1, r1, o2, r2, eps=.1):
    # finds intersection points of 2 circles
    # if no intersection, raises exception
    o1, o2 = numpify(o1, o2)
    d = dist(o1, o2)
    assert d <= r1+r2
    assert d > eps
    a = (r1**2-r2**2+d**2)/(2*d)
    h = (r1**2 - a**2)**.5
    P = (o2-o1)*a/d+o1
    perp_ = perp(o2-o1)
    I1, I2 = P+h*perp_, P-h*perp_
    return I1, I2

def normalize(vec):
    return np.array(vec)/length(vec)

def perp(vec):
    return normalize(vec)[::-1]*np.array([-1,1])

def length(vec):
    return dist(vec, [0]*len(vec))

def numpify(*args):
    arrays = [0]*len(args)
    for i, arg in enumerate(args):
        arrays[i] = arg.astype(np.float32) if isinstance(arg, np.ndarray) else np.array(arg, dtype=np.float32)
    return arrays

def find_circles(A_b, B_b, A_d, B_d):
    # find center and radius of circle, given coords in board frame and dobot frame
    # returns o1, o2, r
    A_b, B_b, A_d, B_d = numpify(A_b, B_b, A_d, B_d)
    a = length(A_d)
    b = length(B_d)
    c = length(A_d - B_d)
    cos_alpha = (a**2+b**2-c**2)/2/a/b
    sin_alpha = (1-cos_alpha**2)**.5
    AB_b = B_b-A_b
    l = length(AB_b)
    r = l/2/sin_alpha
    normalizeal = r*cos_alpha*perp(AB_b)
    M = (A_b+B_b)/2
    o1, o2 = M+normalizeal, M-normalizeal
    return o1, o2, r

def count_coincides(A, points, eps = .1):
    return len([0 for point in points if dist(A, point) < eps])

def is_clockwise(vec1, vec2):
    # finds if vec2 is rotated clockwise from vec1
    cross = vec1[0]*vec2[1] - vec1[1]*vec2[0]
    assert cross != 0
    return cross < 0

def find_origin(board_coords, dobot_coords, j1_positions):
    # find center, given coords of 3 points
    # points all have different angle for 1st joint
    assert len(board_coords) == len(dobot_coords) == 3
    circles = []
    board_coords = list(map(coords_from_chess_pos, board_coords))
    for i, j in combinations([0,1,2], 2):
        o1, o2, r = find_circles(board_coords[i], board_coords[j], dobot_coords[i], dobot_coords[j])
        circles.extend(([o1, r], [o2, r]))
    inters = []
    for circ1, circ2 in combinations(circles, 2):
        inters.extend(find_intersections(*circ1, *circ2))
    centers = [p for p in inters if count_coincides(p, inters) == 3]
    c0 = centers[0]
    eps = .1
    for c in centers:
        if dist(c0, c) > eps:
            c1 = c
            break
    board_coords.append(board_coords[0])
    j1_positions = j1_positions + [j1_positions[0]]
    for center in (c0, c1):
        vectors = [b_point - center for b_point in board_coords]
        for v1, v2, j_val1, j_val2 in zip(vectors[:-1], vectors[1:], j1_positions[:-1], j1_positions[1:]):
            # clockwise rotation is changing j1 angle to lesser angle
            if not is_clockwise(v1, v2) == (j_val2 - j_val1 < 0):
                break
        else:
            return center
    raise ValueError('bad points passed')

def find_rot_matrix(p_b, q_a, origin):
    """
    returns rotation matrix R_ab
    p_b - coordinates of point in board frame
    q_a - coordinates shown by dobot
    origin - coordinates of dobot's origin in board frame
    """
    p_b = coords_from_chess_pos(p_b)
    p_b, q_a, origin = numpify(p_b, q_a, origin)
    # same vector in different frames
    v_b = normalize(p_b-origin)
    v_a = normalize(q_a)
    
    angle = np.arccos(np.inner(v_a, v_b))
    print(f'{angle=}')
    print(f'{v_b=}, {v_a=}')
    angle = (atan2(*v_b) - atan2(*v_a))%(2*np.pi)
    c, s = np.cos(angle), np.sin(angle)
    R_ab = np.array([[c, -s],
                     [s, c]])
    return R_ab

def find_d_l(board_coords, dobot_coords, origin):
    """
    board_coords - coords of 3 points in board's frame
    dobot_coords - coords shown by dobot
    origin - origin of dobot in board's frame
    
    returns d - offset, l - size of tile (both in mm, if given dobot coords are in mm)
    """
    board_coords = list(map(coords_from_chess_pos, board_coords))
    dobot_coords, origin = numpify(dobot_coords, origin)
    lengths_b = [length(point-origin) for point in board_coords]
    lengths_a = [length(point) for point in dobot_coords]
    eps = .1
    print(f'{lengths_b=}, {lengths_a=}')
    for (l_a1, l_b1), (l_a2, l_b2) in combinations(list(zip(lengths_a, lengths_b)), 2):
        if abs(l_b1 - l_b2) > eps:
            l = (l_a1 - l_a2)/(l_b1-l_b2)
    d = l_b1*l - l_a1
    return d, l

In [118]:
class Dobot:
    def __init__(self, api):
        """
        origin - dobot origin in board's coords
        """
        self.api = api
        self.temp = {}
    
    def dobot_xy_from_board_xy(self, board_xy):
        """
        board_xy - can either be like 'b3' or like [1, 2]
        """
        t = self.R_ab@coords_from_chess_pos(board_xy) - self.origin
        return t*self.tile_size + self.offset * normalize(t)
    
    def move_xyz(self, x, y, z, j4_rot=0):
        dType.SetPTPCmd(api,
            dType.PTPMode.PTPMOVLXYZMode,
            x,
            x,
            x,
            j4_rot,
            isQueued=1
        )
    
    def chess_piece_height(piece):
        if piece is None:
            return self.zero_height
        else:
            return self.zero_height + self.piece_height[piece]
    
    def move_to_tile(self, board_xy, chess_piece=None):
        dobot_xy = self.dobot_xy_from_board_xy(board_xy)
        self.move_xyz(*dobot_xy, chess_piece_height(chess_piece))
    
    def get_height(self):
        dType.GetPose(api)[2]
    
    def calibrate_height(self):
        input('Move to zero height')
        self.zero_height = self.get_height()
        pieces = ['pawn', 'rook', 'knight', 'bishop', 'queen', 'king']
        self.pieces_heights = {}
        for piece in pieces:
            input(f"Move to {piece}'s height")
            self.pieces_heights[piece] = self. get_height()
    
    def calibrate_pos(self, chk1=False):
        board_coords = []
        dobot_coords = []
        j1_poses = []
        
        if not chk1:
            for _ in range(3):
                board_coords.append(input('Move to next tile and print its name: '))
                dobot_coords.append(dType.GetPose(api)[:2])
                j1_poses.append(dType.GetPose(api)[4])

            self.temp['bc'] = board_coords
            self.temp['dc'] = dobot_coords
            self.temp['j1'] = j1_poses
        else:
             board_coords = self.temp['bc']
             dobot_coords = self.temp['dc']
             j1_poses = self.temp['j1']
        
        self.origin = find_origin(board_coords, dobot_coords, j1_poses)
        print(f'{self.origin}')
        self.R_ab = find_rot_matrix(board_coords[0], dobot_coords[0], self.origin)
        print(f'{self.R_ab}')
        self.offset, self.tile_size = find_d_l(board_coords, dobot_coords, self.origin)
        print(f'{self.offset}, {self.tile_size}')
    
    def calibrate_image(self, tile_size_px, image_center_px, bottom):
        """
        image_center_px - coords of center of the image. It is half tile lower than board's center
        bottom - where row, where white stand, is
            S, E, N, W
        """
        self.tile_size_px = tile_size_px
        self.image_center_px = np.array(image_center_px)
        if bottom == 'S':
            self.coords_changer = np.array([[1, 0],
                                            [0, -1]])
        if bottom == 'N':
            self.coords_changer = np.array([[-1, 0],
                                            [0, 1]])
        if bottom == 'E':
            self.coords_changer = np.array([[0, -1],
                                            [-1, 0]])
        if bottom == 'W':
            self.coords_changer = np.array([[0, 1],
                                            [1, 0]])
        
    def board_xy_from_px(self, px_xy):
        board_delta = (np.array(px_xy) - self.image_center_px)/self.tile_size_px
        board_delta[1] += .5
        board_coords = self.coords_changer@board_delta + np.array([3.5, 3.5])
        
    def move_to_px(self, px_xy, piece=None):
        self.move_to_tile(self.board_xy_from_px(px_xy), piece)

In [84]:
db = Dobot(api)

In [150]:
db.calibrate_pos(chk1=True)

[ 4.30672744 12.07019155]
angle=2.8543348
v_b=array([ 0.28465262, -0.9586307 ], dtype=float32), v_a=array([-0.00138609,  0.9999991 ], dtype=float32)
[[-0.95902436 -0.28332361]
 [ 0.28332361 -0.95902436]]
lengths_b=[9.461611330967289, 8.07601824340075, 8.278617467664539], lengths_a=[269.3556664640108, 233.73830672044494, 238.075045970199]
-60.86703623480713 21.405507674143518


In [154]:
import scipy.optimize as opt

In [158]:
def find_d_l(d_l, lengths_b, lengths_a):
    errors_sq = [(d_l[1]*l_b-d_l[0]-l_a)**2 for l_b, l_a in zip(lengths_b, lengths_a)]
    return sum(errors_sq)

In [168]:
l_b[2]*l-d

238.57276896968574

In [170]:
d, l

(-23.66266723599232, 25.959660845921558)

In [165]:
l_b, l_a = [9.461611330967289, 8.07601824340075, 8.278617467664539], [269.3556664640108, 233.73830672044494, 238.075045970199]
d, l = opt.minimize(find_d_l, [0, 28], args=(l_b, l_a)).x

In [148]:
print(db.temp)

{'bc': ['h4', 'e5', 'a6'], 'dc': [[-0.3733510971069336, 269.35540771484375], [74.6894302368164, 221.48382568359375], [176.3834991455078, 159.9018096923828]], 'j1': [90.07941436767578, 71.36470794677734, 42.19411849975586]}
